# Create a Grafana Data Source and Dashboard

Follow the steps in this this tutorial to create a "stocks" Grafana dashboard using Iguazio data source <br>
Note that you need to create a Grafana service before running the notebook <br>
Creating a grafana service can be done from the "services" screen in the Platform

In [1]:
import nuclio

## Environment

In [2]:
# %nuclio cmd -c pip install git+https://github.com/v3io/grafwiz --upgrade
# %nuclio cmd -c pip install v3io_frames

## Function

In [3]:
# nuclio: start-code

In [4]:
from grafwiz import *
import os
import v3io_frames as v3f

In [5]:
from grafwiz import *
import v3io_frames as v3f

def handler(grafana_url:str = 'http://grafana-dani', 
                     streamview_url:str = 'http://default-tenant.app.dev8.lab.iguazeng.com:30172',
                     readvector_url:str = 'http://default-tenant.app.dev8.lab.iguazeng.com:30505',
                     v3io_container:str = 'users',
                     stocks_kv_table:str = os.getenv('V3IO_USERNAME') + '/stocks/stocks_kv',
                     stocks_tsdb_table:str = os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb',
                     news_nosql_table:str = "stocks-test-dani/fs/nosql/sets/news-latest"):
    
    
    # Create datasources
    
    DataSource(name='Iguazio').deploy(grafana_url, use_auth=True)
    DataSource(name='stream-viewer', frames_url=streamview_url).deploy(grafana_url, use_auth=False, overwrite=False)
    DataSource(name='read-vector', frames_url=readvector_url).deploy(grafana_url, use_auth=False, overwrite=False)
    
    # Verify the KV table can be shown
    client = v3f.Client('framesd:8081', container=v3io_container)
    try:
        client.execute(backend='kv', table=stocks_kv_table, command='infer')
    except: 
        print('KV Table has no values yet, please run `client.execute(backend="kv", table=stocks_kv_table, command="infer")` (can be found on the 05-explore notebook.)')
    
    # Create grafana dashboard
    dash = Dashboard("stocks", start='now-7d', dataSource='Iguazio')

    # Add a symbol combo box (template) with data from the stocks table
    dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=" + os.getenv('V3IO_USERNAME') + "/stocks/stocks_kv;backend=kv;container=users")

    # Create a table and log viewer in one row
    tbl = Table('Current Stocks Value', span=12).source(table=stocks_kv_table,fields=['symbol','volume', 'price', 'sentiment', 'last_updated'],container=v3io_container)
    dash.row([tbl])
    
    # Create 2 charts on the second row
    metrics_row = [Graph(metric).series(table=stocks_tsdb_table, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']]
    metrics_row.append(Graph('sentiment').series(table=stocks_tsdb_table, fields=['sentiment'], filter='symbol=="$SYMBOL"', container=v3io_container))
    dash.row(metrics_row)
    
    # Create log veiwer panel
    log = Table('Articles Log', dataSource='stream-viewer', span=12)
    dash.row([log])
    log = Table('Vector Log', dataSource='read-vector', span=12)
    dash.row([log])
    
    # Deploy to Grafana
    return dash.deploy(grafana_url)

In [6]:
# nuclio: end-code

## Local test

In [26]:
from mlrun import code_to_function
fn = code_to_function('grafana_display',
                      handler='handler',kind="job")

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : 'stocks-test-dani',
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [25]:
fn.deploy_step()
s = fn.as_step(inputs = {"daniel":"sabba"})
fn.deploy()

> 2021-03-24 11:47:31,782 [info] starting remote build, image: .mlrun/func-default-grafana-display-latest
E0324 11:48:13.909903       1 aws_credentials.go:77] while getting AWS credentials NoCredentialProviders: no valid providers in chain. Deprecated.
	For verbose messaging see aws.Config.CredentialsChainVerboseErrors
INFO[0040] Retrieving image manifest mlrun/mlrun:0.6.2-rc1 
INFO[0043] Retrieving image manifest mlrun/mlrun:0.6.2-rc1 
INFO[0046] Built cross stage deps: map[]                
INFO[0046] Retrieving image manifest mlrun/mlrun:0.6.2-rc1 
INFO[0048] Retrieving image manifest mlrun/mlrun:0.6.2-rc1 
INFO[0051] Executing 0 build triggers                   
INFO[0051] Unpacking rootfs as cmd RUN pip install git+https://github.com/v3io/grafwiz --upgrade requires it. 
INFO[0077] RUN pip install git+https://github.com/v3io/grafwiz --upgrade 
INFO[0077] Taking snapshot of full filesystem...        


KeyboardInterrupt: 

In [7]:
from mlrun import code_to_function, run_local

fn = code_to_function('grafana_dashboard_deployer',
                      kind='job',
                      handler='deploy_dashboard',
                      project='stocks')
fn.spec.build.base_image = 'mlrun/mlrun'
fn.export('04-grafana.yaml')

> 2021-03-22 10:09:47,084 [info] function spec saved to path: 04-grafana.yaml


In [8]:
run_local(handler=deploy_dashboard)

> 2021-03-22 10:09:49,019 [info] starting run mlrun-dd8d2c-deploy_dashboard uid=2a5f287071a84818b9af2c2cf115f5cb DB=http://mlrun-api:8080
Datasource Iguazio already exists
Datasource Iguazio created successfully
Datasource stream-viewer already exists
Datasource stream-viewer created successfully
Dashboard stocks created successfully


AttributeError: 'Series' object has no attribute '_mgr'